In [1]:
import numpy as np

In [2]:
import numpy as np

# Define the domain
x = np.linspace(0, 2, 1000)  # 1000 points between 0 and 2

# Define the function to be integrated
y = x  # since we are integrating x

# Perform the integration using the trapezoidal rule
integral = np.trapz(y, x)

print(f"The integral of x from 0 to 2 is approximately {integral}")


The integral of x from 0 to 2 is approximately 2.0


In [5]:
import numpy as np

# Define the function to be integrated
def f(x, y):
    return x * y

# Define the domain for x and y
x = np.linspace(0, 2, 100)
y = np.linspace(0, 3, 100)

# Create a grid of points
X, Y = np.meshgrid(x, y)

# Evaluate the function on the grid
Z = f(X, Y)

# Perform the double integration using the trapezoidal rule
# First, integrate with respect to y for each fixed x
integral_y = np.trapz(Z, y, axis=0)

# Then, integrate the result with respect to x
integral = np.trapz(integral_y, x)

print(f"The double integral of xy over [0, 2] x [0, 3] is approximately {integral}")


The double integral of xy over [0, 2] x [0, 3] is approximately 9.0


In [ ]:
def GF_table(x, case_x: str, case_y: str, L):
    # returns the eigenfunc(x) Xm, the eigenvalue betam and the norm Nx

    m = np.arange(1, len(x))
    if (case_x == 'D' and case_y == 'D'): #11
        beta_m = m*np.pi/L
        X_m = np.sin(beta_m*x)
        Nx = L/2
        return X_m, beta_m, Nx
    
    if (case_x == 'D' and case_y == 'N'): #12
        beta_m = (2*m-1)*np.pi/(2*L)
        X_m = np.sin(beta_m*x)
        Nx = L/2
        return X_m, beta_m, Nx
    

def green_function(x, y, xs, ys, case_n: str, case_s: str):

